In [1]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import time
import codecs
import pandas as pd
import xml.etree.ElementTree as ET
import pymarc
from pymarc import Record,Field
import marcx
import io
from lxml import etree
import json
#from spst import SPST
import os
os.chdir('/Users/jammerman/recover')

def line2dict(line):
    columns = ['Volume Identifier','Access','Rights','HathiTrust record number','Enumeration/Chronology','Source',
           'Source institution record number','OCLC numbers','ISBNs','ISSNs','LCCNs','Title','Imprint',
           'Rights determination reason code','Date of last update','Government Document','Publication Date',
          'Publication Place','Language','Bibliograhic Format']
    columns = ['Location Code','Display Call Number','Display Call Number Normalized','Title','Enumeration',
               'Copy Number','Possible Duplicate','Author','Publisher','Publication Year','Edition',
               'Primary Language','Series','Recorded Uses - Item','Recorded Uses - Title','Last Charge Date - Item',
               'Last Charge Date - Title','Last Add Date - Title','Global Holdings - Same Edition',
               'US Holdings - Same Edition','US Holdings - Any Edition','Massachusetts Holdings - Same Edition',
               'Massachusetts Holdings - Any Edition','EAST Overlap','All Comparator Library Holdings - Same Edition',
               'All Comparator Library Holdings - Any Edition','Maine Shared Collections - Same Edition',
               'Maine Shared Collections - Any Edition','Regional Large Academic Libraries - Same Edition',
               'Regional Large Academic Libraries - Any Edition','Regional College Libraries - Same Edition',
               'Regional College Libraries - Any Edition','Boston Library Consortium (BLC) - Same Edition',
               'Boston Library Consortium (BLC) - Any Edition','CNY Retention Partners - Same Edition',
               'CNY Retention Partners - Any Edition','Ephemera','HathiTrust Public Domain',
               'HathiTrust In Copyright','HathiTrust URL','OPAC URL','WorldCat URL','OCLC Work ID',
               'WorldCat OCLC Number','Bib Record Number','Bib Control Number','Item Control Number',
               'Item Type Code','Item Status Code','ISBN','Barcode']
    line = line.replace('\n','').split('\t')
    d = {}
    %pdb
    for i in range(0,len(columns)):
        d[columns[i]] = line[i]
    return(d)

#f = open('Boston_University_-_In_HathiTrust_-_2016-03-29_090304.csv','r')
df = pd.read_csv('Boston_University_-_In_HathiTrust_-_2016-03-29_090304.csv',low_memory=False)
#df.head()

In [2]:
df = df[ ['Title','Author','Publisher','Publication Year','Edition','Recorded Uses - Title',
                       'HathiTrust Public Domain','HathiTrust In Copyright','HathiTrust URL','OPAC URL',
                       'WorldCat URL','OCLC Work ID','WorldCat OCLC Number','Bib Record Number',
                       'Bib Control Number','Item Control Number','ISBN']]

In [3]:
df = df.set_index('Bib Record Number')
df.shape

(387407, 16)

In [4]:
def get_representation(mms_id):
    '''get_representation:
    Parameter: an mms_id
    returns a dictionary containing:
    the id for a digital representation attached to the bib record if the representation exists
    the mms_id
    the oai record identifer
    '''
    url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/representations'.replace('{mms_id}',quote_plus(mms_id))
    queryParams = '?' + urlencode({ quote_plus('apikey') : 'l7xxf2bd5e66223d4008b04eba8751a96856'  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read().decode()
    root = ET.fromstring(response_body)
    rep_xml = ''
    d = {}
    d['mms_id'] = mms_id
    d['originating_record_id'] = 'none'
    d['id'] = 'none'
    try:
        for kid in root[0].getchildren():
            d[kid.tag] = kid.text.replace('%3A',':').replace('%2F','/')
    except Exception as e:
        d['id'] = 'none'
        pass
    if '1161' in d['id']:
        #print('Found it: ',d['id'])
        
        url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/representations/{rep_id}'.replace('{mms_id}',quote_plus(mms_id)).replace('{rep_id}',quote_plus(d['id']))
        queryParams = '?' + urlencode({ quote_plus('apikey') : 'l7xxf2bd5e66223d4008b04eba8751a96856'  })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        rep_xml = urlopen(request).read().decode()
        #print('get_reps: ',d)
    return(d,rep_xml)  

def add_representation(mms_id,identifier,rights):
    '''add_representation adds a digital reprentation record to a bib record in Alma for a
    digital object residing in an institutional repository
    Parameters: mms_id, the OAI record identifier
    Returns the mms_id, the OAI record identifier, and the ID for the digital representation'''
    if rights == 'pd':
        rights = 'Public Domain : You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission.'
    if rights == 'pdus':
        rights = "Public Domain (US) : You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission in the U.S."
    if rights == 'cc-by-nc-nd-3.0':
        rights += 'This work is protected by copyright law,but made available under a Creative Commons Attribution-NonCommercial-NoDerivatives license.'
    if rights == 'cc-by-nc-nd-4.0':
        rights += 'This work is protected by copyright law,but made available under a Creative Commons Attribution-NonCommercial-NoDerivatives license.'
    if rights == 'cc-by-nc-3.0':
        rights += 'This work is protected by copyright law,but made available under a Creative Commons Attribution-NonCommercial license.'
    if rights == 'cc-by-nc-4.0':
        rights += 'This work is protected by copyright law,but made available under a Creative Commons Attribution-NonCommercial-ShareAlike license.'
    if rights == 'cc-by-nc-sa-4.0':
        rights += 'This work is protected by copyright law,but made available under a Creative Commons Attribution-NonCommercial-ShareAlike license.'
        
    delivery_url = identifier.replace('%3A',':').replace('%2F','/') #identifier[identifier.rindex(':')+1:]
    linking_parameter = identifier[37:] #identifier[identifier.rindex(':')+1:]
    url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/representations'.replace('{mms_id}',quote_plus(mms_id))
    #queryParams = '?' + urlencode({ quote_plus('apikey') : 'l7xx4f9174fae8bc49a780ba6d5a93151af0'  }) ## sandbox
    queryParams = '?' + urlencode({ quote_plus('apikey') : 'l7xxf2bd5e66223d4008b04eba8751a96856'  }) ## production
    values  = '''<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
             <representation is_remote="true">    
             <id />    
             <library desc="Mugar">MUG</library>    
             <usage_type desc="Derivative">DERIVATIVE_COPY</usage_type> 
             <public_note>{rights}</public_note>
             <delivery_url>{delivery_url}</delivery_url>    
             <thumbnail_url/>    
             <repository desc="HathiTrust">HathiTrust</repository>  
             <originating_record_id>Record/{identifier}</originating_record_id>    
             <linking_parameter_1>{linking_parameter}</linking_parameter_1>    
             <linking_parameter_2/>    
             <linking_parameter_3/>    
             <linking_parameter_4/>    
             <linking_parameter_5/>    
             <created_by>jwasys</created_by>    
             <created_date>2016-01-29Z</created_date>    
             <last_modified_by>jwasys</last_modified_by>    
             <last_modified_date>2016-01-219Z</last_modified_date>
             </representation>'''
    values = values.replace('{mms_id}',quote_plus(mms_id)).replace('{identifier}',quote_plus(linking_parameter))
    values = values.replace('{rights}',rights)
    values = values.replace('{linking_parameter}',quote_plus(linking_parameter)).replace('{delivery_url}',quote_plus(delivery_url)).encode("utf-8")
    headers = {  'Content-Type':'application/xml'  }
    request = Request(url + queryParams
    , data=values
    , headers=headers)
    request.get_method = lambda: 'POST'
    response_body = urlopen(request).read()
    tree = etree.fromstring(response_body)
    x = tree.find('id')
    return(mms_id,identifier,x.text)

def update_bib(mms_id,ht_recno,rights):
    
    ## first get the current bib 
    url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}'.replace('{mms_id}',quote_plus(mms_id))
    queryParams = '?' + urlencode({ quote_plus('expand') : 'None' ,quote_plus('apikey') : 'l7xxf2bd5e66223d4008b04eba8751a96856'  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    #%pdb
    
    ## now let's add the ht_recno to 024$a
    rights = rights.replace('pd','public domain').replace('pdus','public domain (US)').replace('icworld','in copyright')
    _024 = '''       <datafield ind1=" 7" ind2=" " tag="024">
            <subfield code="a">{{ht_record}}</subfield>
            <subfield code="c">{{rights}}</subfield>
            <subfield code="2">HathiTrust</subfield>
            <subfield code="0">http://catalog.hathitrust.org/Record/{{ht_record}}</subfield>
        </datafield>'''.replace('{{ht_record}}',ht_recno).replace('{{rights}}',rights)
    field = etree.fromstring(_024)
    bib = etree.fromstring(response_body)
    field024=bib.findall('./record/datafield/[@tag="024"]')
    #print(len(field024))
    for i in field024:
        suba = i.find('*/[@code="a"]')
        sub2 = i.find('*/[@code="2"]')
        #print(sub2.text)
        if sub2.text == 'HathiTrust':
            return(response_body)
    record = bib.find('./record')
    record.append(field)
    
    url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}'.replace('{mms_id}',quote_plus(mms_id))
    queryParams = '?' + urlencode({ quote_plus('apikey') : 'l7xxf2bd5e66223d4008b04eba8751a96856'  })
    values  = ET.tostring(bib)
    headers = {  'Content-Type':'application/xml'  }
    request = Request(url + queryParams, data=values, headers=headers)
    request.get_method = lambda: 'PUT'
    response_body = urlopen(request).read()
    return(response_body)

In [6]:
count = 0
for i,r in df[:200].iterrows():
    time.sleep(1)
    mms_id = str(i)
    ht_recno = r['HathiTrust URL'][37:]
    ht_bib = 'http://catalog.hathitrust.org/api/volumes/brief/recordnumber/<id value>.json'.replace('<id value>',ht_recno)
    try:
        json_str = json.loads(urlopen(ht_bib).read().decode().replace('\n',''))
        record = json_str['records']
        items = json_str['items'][0]
        rights = items['usRightsString']
        rightscode = items['rightsCode']
    except Exception as e:
        continue
    bib_rec=update_bib(mms_id,ht_recno,rightscode)
    #print(record)
    #print(items)
    if rights != 'Full view':
        print(count + 1)
    else:
        print(str(count),mms_id,ht_recno,rights,rightscode)
        count += 1
        ## check to see if there is an existing digital representation
        reps = get_representation(mms_id)
        ##     create a digital representation using the HathiTrust URL because there are none
        if (reps[0]['id']=='none'):
            add_representation(mms_id,r['HathiTrust URL'],rightscode)
        ##  don't   create a digital representation using the HathiTrust URL because there is one
        elif reps[1].index(r['HathiTrust URL'])> 0:
            print(reps[1].index(r['HathiTrust URL'])> 0)
        ##     create a digital representation using the HathiTrust URL because there are none
        else:
            add_representation(mms_id,r['HathiTrust URL'],rightscode)
        

0 99178803900001161 009788558 Full view pd
True
1 99189419360001161 009788559 Full view pd
True
2 99189483740001161 001015976 Full view pd
True
3 99188032780001161 004553225 Full view pd
True
4 99175028680001161 003122051 Full view pd
True
5 99179199380001161 004462939 Full view pd
True
6 99189416400001161 100558203 Full view pd
True
7 99189477380001161 000249877 Full view pd
True
8 99190428140001161 001013853 Full view pd
True
9 99184491200001161 001017149 Full view pd
True
10 99177945510001161 004553234 Full view pd
True
11 99194264910001161 001473210 Full view pdus
True
12 99195151850001161 001359696 Full view pd
True
13 99190426960001161 009788567 Full view pd
True
14 99178987810001161 009788569 Full view pd
True
15 99183529120001161 001473211 Full view pd
True
16 99189421350001161 000622045 Full view pd
True
17 99189429340001161 009788583 Full view pd
True
18 99189429340001161 009788583 Full view pd
True
19 99192087100001161 001017015 Full view pd
True
20 99192087100001161 0010170